# 12-02 Practice 1: Web(블로그)

### 환경설정


API KEY 를 설정합니다.


In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
!pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH12-RAG-practice")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH12-RAG-practice


### 블로그

In [ ]:
## 추가된 코드
#%pip install --upgrade "pydantic>=2.7.4" langchain langchain-openai

In [11]:
import bs4 
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
# URL 설정
url = "https://m.blog.naver.com/hongcouple/221793269584"

# 단계 1: 문서 로드 (Load Documents)
loader = WebBaseLoader(
    web_paths=(url,),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer("div"))  # 모든 div 태그의 텍스트 가져오기
)

docs = loader.load()

# ✅ 문서가 비어 있으면 예외 발생
if not docs:
    raise ValueError("문서를 불러오지 못했습니다. URL 또는 HTML 구조를 확인하세요.")

# 문서 로드 결과 확인
print(f"불러온 문서 개수: {len(docs)}")
if docs:
    print(f"첫 번째 문서 내용 일부: {docs[0].page_content[:500]}")

# 단계 2: 문서 분할 (Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

# ✅ 문서 분할 결과가 비어 있으면 예외 발생
if not splits:
    raise ValueError("문서 분할 결과가 비어 있습니다. 크롤링된 데이터 확인 필요.")

# 문서 분할 결과 확인
print(f"분할된 문서 개수: {len(splits)}")
if splits:
    print(f"첫 번째 청크 내용: {splits[0].page_content[:500]}")

# 단계 3: 임베딩 & 벡터스토어 생성 (Create Vectorstore)
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 단계 4: 검색(Search)
retriever = vectorstore.as_retriever()

# 단계 5: 프롬프트 생성 (Create Prompt)
try:
    import langchain
    prompt = langchain.hub.pull("rlm/rag-prompt")
except Exception as e:
    raise RuntimeError(f"프롬프트를 로드하는 데 실패했습니다: {e}")

# 단계 6: 언어모델 생성 (Create LLM)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    """ 검색한 문서를 하나의 문단으로 합침 """
    return "\n\n".join(doc.page_content for doc in docs)


# 단계 7: 체인 생성 (Create Chain)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 단계 8: 체인 실행 (Run Chain)
question = "공리주의에 대해 설명해 주세요"

try:
    response = rag_chain.invoke(question)
except Exception as e:
    response = f"응답을 생성하는 데 실패했습니다: {e}"

# 결과 출력
print(f"URL: {url}")
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")


불러온 문서 개수: 1
첫 번째 문서 내용 일부: 




본문 바로가기


블로그





카테고리 이동




hongfamily의 서재






검색
MY메뉴 열기

















서평


[서평] 정의란 무엇인가 - JUSTICE / 마이클 샌델 /MICHAEL J. SANDEL 






hongfamily

2020. 2. 2. 9:20
	


 이웃추가


본문 기타 기능



본문 폰트 크기 조정
본문 폰트 크기 작게 보기
본문 폰트 크기 크게 보기
가






 공감하기





공유하기 


URL복사 
신고하기 


















 

 



이 책은 중학생인 아들과 함께 읽고 의견을 나누어보고자 선택한 책인데, 정의라는 다소 형이상학적이고 철학적인 주제를 다루고 있으면서도, 실생활에서 접하게 되는 실질적인 문제들을 토론의 주제로 삼음으로써 독자를 몰입하게 만드는 힘이 있었다. 서점에서 책을 훑어보다가 일독을 하고 싶을 정도였다면 믿겠는가.​이 책을 읽다 보면 하버드 대학의 강
분할된 문서 개수: 11
첫 번째 청크 내용: 본문 바로가기


블로그





카테고리 이동




hongfamily의 서재






검색
MY메뉴 열기

















서평


[서평] 정의란 무엇인가 - JUSTICE / 마이클 샌델 /MICHAEL J. SANDEL 






hongfamily

2020. 2. 2. 9:20
	


 이웃추가


본문 기타 기능



본문 폰트 크기 조정
본문 폰트 크기 작게 보기
본문 폰트 크기 크게 보기
가






 공감하기





공유하기 


URL복사 
신고하기
URL: https://m.blog.naver.com/hongcouple/221793269584
문서의 수: 1
[HUMAN]
공리주의에 대해 설명해 주세요

[AI]
공리주의는 제레미 벤담이 주창한 이론으로 도덕의 최고 원칙은 행복의 극대화, 즉 쾌락의 총량이 고통의 총량보다 많게 하는 데 있다고